In [2]:
# from traj_len_15 import DeepAccidentDataset
from data.Safe2Unsafe import DeepAccidentDataset
from method.ViT import InDCBFTrainer, InDCBFController, Barrier
import torch
import time
import pytorch_lightning as pl

In [5]:
data = DeepAccidentDataset(train_batch_size=32,val_batch_size=32,num_workers=16)
data.setup()
train_dataloader = data.train_dataloader()
test_dataloader = data.val_dataloader()
barrier = Barrier(2,512)
model = InDCBFController(2,"cuda",model="google/vit-base-patch16-224",latent_dim=512)
checkpoint = torch.load("/root/tf-logs/BarrierClassifier/version_1/checkpoints/epoch=80-step=16037.ckpt")
trainer = InDCBFTrainer(model,barrier)
trainer.load_state_dict(checkpoint['state_dict'])

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [6]:
for idx, (i_all,u_all,label) in enumerate(test_dataloader):
    if label.sum()>0:
        break

In [15]:
x_init = torch.zeros(1,512).to('cuda')
model = trainer.model.to("cuda")
barrier = trainer.barrier.to("cuda")
print(label)

tensor([[1],
        [2],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0]])


In [35]:
imgs = i_all[10].to("cuda")
us = u_all[10].to("cuda")
x_p = model.encoder(imgs[0,:].unsqueeze(0),x_init,us[0].unsqueeze(0))
for i in range(1,15):
# u, result = model(i[0,1].unsqueeze(0),u[0,0].unsqueeze(0),x_p,u[0,0].unsqueeze(0),barrier)
    v, result, prob = model(imgs[i].unsqueeze(0),us[i-1].unsqueeze(0),x_p,us[i].unsqueeze(0),barrier)
    x_p = model.encoder(imgs[i,:].unsqueeze(0),x_p,us[i-1].unsqueeze(0))
    print(v.value,us[i])
# print(us[0])
# print(us[1])

[ 0.0508374  -4.17871857] tensor([ 0.0508, -4.1787], device='cuda:0')
[ 0.04569366 -3.84402585] tensor([ 0.0457, -3.8440], device='cuda:0')
[ 0.04593947 -3.83066034] tensor([ 0.0459, -3.8307], device='cuda:0')
[ 0.04822314 -3.99490833] tensor([ 0.0482, -3.9949], device='cuda:0')


IndexError: index 5 is out of bounds for dimension 0 with size 5

In [29]:
print(v.value)
print(us[10])

[-2506.46806297  -386.10911042]


IndexError: index 10 is out of bounds for dimension 0 with size 5

In [37]:
u_ref = us[1].unsqueeze(0)
u_p = us[1].unsqueeze(0)
i = imgs[1].unsqueeze(0)
import cvxpy as cp
u_ref = u_ref.view(-1).cpu().numpy()
x = model.encoder(i,x_p,u_p)
f, g = model.ode(x)
f = f.view(-1).detach().cpu().numpy()
g = g.view(-1,2).detach().cpu().numpy()
b = barrier(x)
d_b = torch.autograd.grad(b,x,retain_graph=True)[0]
b = b.view(-1).detach().cpu().numpy()
x = x.view(-1).detach().cpu().numpy()
d_b = d_b.view(-1).cpu().numpy()
u = cp.Variable(u_ref.shape)
t1 = d_b @ f
t2 = d_b @ g 
t3 = b/15
objective = cp.Minimize(cp.sum_squares(u - u_ref))
constraints = [(t1+t2@u+t3)>=0]
prob = cp.Problem(objective, constraints)
result = prob.solve()
t1, t2, t3

(0.000443073,
 array([ 0.00862157, -0.03043856], dtype=float32),
 array([0.0392131], dtype=float32))

(0.0023901234,
 array([-0.00282371, -0.01853349], dtype=float32),
 array([-0.0297253], dtype=float32))

In [7]:
b_all = []
label_all = []
for idx, (i,u,label) in enumerate(test_dataloader):
    i, u = i.to("cuda"), u.to("cuda")
    x,x_tide = model.simulate(i,u,dt=0.1,window_size=1,rtol=5e-6)
    b = barrier(x).squeeze(-1)
    b_all.append(b.cpu())
    label_all.append(label.squeeze(-1))

100%|██████████| 14/14 [00:02<00:00,  4.70it/s]


In [12]:
import numpy as np
bs = torch.cat(b_all)
labels = torch.cat(label_all)
results = torch.cat([bs,labels.unsqueeze(-1)],dim=-1).detach().numpy()
np.savetxt("./results.txt",results)

In [43]:
b = results[:,:-1]
label = results[:,-1]
mask = np.zeros_like(b)
mask[label == 1,-5:] = 1
b_safe = b[label == 0]
b_unsafe = b[label == 1,-5:]

In [44]:
max_safe = b_safe.max()
min_safe = b_safe.min()
mean_safe = b_safe.mean()
std_safe = b_safe.std()
max_unsafe = b_unsafe.max()
min_unsafe = b_unsafe.min()
mean_unsafe = b_unsafe.mean()
std_unsafe = b_unsafe.std()
print("Safe", max_safe, min_safe, mean_safe, std_safe)
print("Unsafe", max_unsafe, min_unsafe, mean_unsafe, std_unsafe)

Safe 1.0 -0.67952687 0.97396356 0.14432254
Unsafe 1.0 -0.6665393 0.91301495 0.343813


In [4]:
b_all_train = []
label_all_train = []
for idx, (i,u,label) in enumerate(train_dataloader):
    i, u = i.to("cuda"), u.to("cuda")
    x,x_tide = model.simulate(i,u,dt=0.1,window_size=1,rtol=5e-6)
    b = barrier(x).squeeze(-1)
    b_all_train.append(b.cpu())
    label_all_train.append(label.squeeze(-1))

100%|██████████| 14/14 [00:02<00:00,  4.69it/s]


In [5]:
import numpy as np
bs_train = torch.cat(b_all_train)
labels_train = torch.cat(label_all_train)
results_train = torch.cat([bs_train,labels_train.unsqueeze(-1)],dim=-1).detach().numpy()
np.savetxt("./results_train.txt",results_train)

In [6]:
b = results_train[:,:-1]
label = results_train[:,-1]
mask = np.zeros_like(b)
mask[label == 1,-5:] = 1
b_safe = b[label == 0]
b_unsafe = b[label == 1,-5:]

In [7]:
max_safe = b_safe.max()
min_safe = b_safe.min()
mean_safe = b_safe.mean()
std_safe = b_safe.std()
max_unsafe = b_unsafe.max()
min_unsafe = b_unsafe.min()
mean_unsafe = b_unsafe.mean()
std_unsafe = b_unsafe.std()
print("Safe", max_safe, min_safe, mean_safe, std_safe)
print("Unsafe", max_unsafe, min_unsafe, mean_unsafe, std_unsafe)

Safe 1.0 -0.60138047 0.9851279 0.0887461
Unsafe 0.9999999 -0.7876854 0.76094234 0.52417785


In [17]:
from data.Safe2Unsafe import DeepAccidentDataset
from method.ResNet import InDCBFTrainer, InDCBFController, Barrier
import torch
import time
import pytorch_lightning as pl
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [14]:
data = DeepAccidentDataset(train_batch_size=32,val_batch_size=32,num_workers=16)
data.setup()
train_dataloader = data.train_dataloader()
test_dataloader = data.val_dataloader()

In [3]:
latent_dim = 16
barrier = Barrier(2,latent_dim=latent_dim)
model = InDCBFController(2,"cuda",model="resnet50",latent_dim=latent_dim)
# model = InDCBFController(2,"cuda",model="google/vit-base-patch16-224",latent_dim=latent_dim)
# model = InDCBFController(2,"cuda",model="openai/clip-vit-base-patch16",latent_dim=latent_dim)
checkpoint = torch.load("/root/tf-logs/InDCBF/version_2/checkpoints/last.ckpt")
trainer = InDCBFTrainer(model,barrier)
trainer.load_state_dict(checkpoint['state_dict'])
trainer.to("cuda")

InDCBFTrainer(
  (model): InDCBFController(
    (encoder): Encoder(
      (ResNet): ResNet(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=T

In [18]:
from method.dynamics import InDCBFDynamics
from method.barriers import InDCBFBarrier
from method.trainers import InDCBFTrainer
latent_dim = 16
barrier = InDCBFBarrier(2,latent_dim=latent_dim)
model = InDCBFDynamics(2,"cuda",model="openai/clip-vit-base-patch16",latent_dim=latent_dim)
# model = InDCBFDynamics(2,"cuda",model="google/vit-base-patch16-224",latent_dim=latent_dim)
trainer = InDCBFTrainer(model,barrier)
checkpoint = torch.load("/root/tf-logs/InDCBFWORec/version_1/checkpoints/last.ckpt")
trainer.load_state_dict(checkpoint['state_dict'])
trainer.to("cuda")

InDCBFTrainer(
  (model): InDCBFDynamics(
    (vae): ViTEncoder(
      (ViT): CLIPVisionModel(
        (vision_model): CLIPVisionTransformer(
          (embeddings): CLIPVisionEmbeddings(
            (patch_embedding): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16), bias=False)
            (position_embedding): Embedding(197, 768)
          )
          (pre_layrnorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (encoder): CLIPEncoder(
            (layers): ModuleList(
              (0): CLIPEncoderLayer(
                (self_attn): CLIPAttention(
                  (k_proj): Linear(in_features=768, out_features=768, bias=True)
                  (v_proj): Linear(in_features=768, out_features=768, bias=True)
                  (q_proj): Linear(in_features=768, out_features=768, bias=True)
                  (out_proj): Linear(in_features=768, out_features=768, bias=True)
                )
                (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affi

In [19]:
b_all = []
label_all = []
# trainer.eval()
for idx, (i,u,label) in enumerate(test_dataloader):
    i, u = i.to("cuda"), u.to("cuda")
    x,x_tide = trainer.model.simulate(i,u,dt=0.1,window_size=1,rtol=5e-6)
    b = trainer.barrier(x).squeeze(-1)
    b_all.append(b.cpu())
    label_all.append(label.squeeze(-1))
import numpy as np
bs = torch.cat(b_all)
labels = torch.cat(label_all)
results = torch.cat([bs,labels.unsqueeze(-1)],dim=-1).detach().numpy()
np.savetxt("./results_clip_multi_rep.txt",results)

100%|██████████| 4/4 [00:00<00:00,  4.33it/s]


In [8]:
b_all_ascent = []
b_all_descent = []
label_all = []
# trainer.eval()
for idx, (i,u,label) in enumerate(test_dataloader):
    label = label.squeeze(dim=-1)
    i, u = i.to("cuda"), u.to("cuda")
    x,x_tide = trainer.model.simulate(i,u,dt=0.1,window_size=1,rtol=5e-6)
    x_safe = x[label == 0]
    x_unsafe = x[label == 1]
    # x_safe.requires_grad = True
    # x_unsafe.requires_grad = True
    b_safe = trainer.barrier(x_safe)
    b_unsafe = trainer.barrier(x_unsafe)

    d_b_safe = torch.autograd.grad(b_safe.mean(),x_safe,retain_graph=True)[0]
    with torch.no_grad():
        f, g = trainer.model.ode(x_safe)
    gu = torch.einsum('btha,bta->bth',g.view(g.shape[0],g.shape[1],f.shape[-1],2),u[label == 0])
    ascent_value = torch.einsum('bth,bth->bt', d_b_safe, (f + gu))
    b_ascent = ascent_value.unsqueeze(-1) + b_safe

    d_b_unsafe = torch.autograd.grad(b_unsafe.mean(),x_unsafe,retain_graph=True)[0]
    with torch.no_grad():
        f, g = trainer.model.ode(x_unsafe)
    gu = torch.einsum('btha,bta->bth',g.view(g.shape[0],g.shape[1],f.shape[-1],2),u[label == 1])
    descent_value = torch.einsum('bth,bth->bt', d_b_unsafe, (f + gu))
    b_descent = descent_value.unsqueeze(-1) + b_unsafe

    b_all_ascent.append(b_ascent.cpu())
    b_all_descent.append(b_descent.cpu())
    label_all.append(label)
import numpy as np
bsa = torch.cat(b_all_ascent)
bsd = torch.cat(b_all_descent)
labels = torch.cat(label_all)
results1 = torch.cat([bsa.squeeze(-1),labels[labels == 0].unsqueeze(-1)],dim=-1).detach().numpy()
results2 = torch.cat([bsd.squeeze(-1),labels[labels == 1].unsqueeze(-1)],dim=-1).detach().numpy()
np.savetxt("./results_grad_asc_resnet.txt",results1)
np.savetxt("./results_grad_dsc_resnet.txt",results2)

100%|██████████| 4/4 [00:01<00:00,  3.12it/s]


RuntimeError: shape '[32, 5, 16, 2]' is invalid for input of size 30720

In [12]:
b_all_ascent = []
b_all_descent = []
label_all = []
# trainer.eval()
for idx, (i,u,label) in enumerate(test_dataloader):
    label = label.squeeze(dim=-1)
    i, u = i.to("cuda"), u.to("cuda")
    x,x_tide = trainer.model.simulate(i,u,dt=0.1,window_size=1,rtol=5e-6)
    x_safe = x[label == 0]
    x_unsafe = x[label == 1]
    # x_safe.requires_grad = True
    # x_unsafe.requires_grad = True

    b_safe = trainer.barrier(x_safe)
    d_b_safe = torch.autograd.grad(b_safe.mean(),x_safe,retain_graph=True)[0]
    with torch.no_grad():
        f, g = trainer.model.ode(x_safe)
    gu = torch.einsum('btnha,btna->btnh',g.view(g.shape[0],g.shape[1],g.shape[2],f.shape[-1],2),u[label == 0].unsqueeze(2).expand(-1,-1,g.shape[2],-1))
    ascent_value = torch.einsum('btnh,btnh->bt', d_b_safe, (f + gu))
    b_ascent = ascent_value.unsqueeze(-1) + b_safe

    b_unsafe = trainer.barrier(x_unsafe)
    print(b_unsafe)
    d_b_unsafe = torch.autograd.grad(b_unsafe.mean(),x_unsafe,retain_graph=True,allow_unused=True)[0]
    with torch.no_grad():
        f, g = trainer.model.ode(x_unsafe)
    gu = torch.einsum('btnha,btna->btnh',g.view(g.shape[0],g.shape[1],g.shape[2],f.shape[-1],2),u[label == 1].unsqueeze(2).expand(-1,-1,g.shape[2],-1))
    descent_value = torch.einsum('btnh,btnh->bt', d_b_unsafe, (f + gu))
    b_descent = descent_value.unsqueeze(-1) + b_unsafe

    b_all_ascent.append(b_ascent.cpu())
    b_all_descent.append(b_descent.cpu())
    label_all.append(label)
import numpy as np
bsa = torch.cat(b_all_ascent)
bsd = torch.cat(b_all_descent)
labels = torch.cat(label_all)
results1 = torch.cat([bsa.squeeze(-1),labels[labels == 0].unsqueeze(-1)],dim=-1).detach().numpy()
results2 = torch.cat([bsd.squeeze(-1),labels[labels == 1].unsqueeze(-1)],dim=-1).detach().numpy()
np.savetxt("./results_grad_asc_vit_multi_rep.txt",results1)
np.savetxt("./results_grad_dsc_vit_multi_rep.txt",results2)

100%|██████████| 4/4 [00:01<00:00,  2.99it/s]


tensor([], device='cuda:0', size=(0, 5, 1), grad_fn=<TanhBackward0>)


TypeError: expected Tensor as element 0 in argument 1, but got NoneType

In [21]:
asc = np.loadtxt("./results_grad_asc_vit.txt")[:,:-1]
dsc = np.loadtxt("./results_grad_dsc_vit.txt")[:,:-1]
acc_regular = (asc > 0).mean()
acc_collision = (dsc < 0).mean()
print(acc_regular,acc_collision)

0.9585430463576159 0.5733333333333334


In [20]:
results = np.loadtxt("./results_clip_multi_rep.txt")
regular = results[results[:,-1] == 0,:-1]
collision = results[results[:,-1] == 1,:-1]

In [21]:
acc_regular = (regular > 0).mean()
acc_collision = (collision < 0).mean()
print(acc_regular,acc_collision)

0.9031788079470199 0.56


In [3]:
from diffusers.models import AutoencoderKL
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-mse")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


In [4]:
from PIL import Image
from torchvision.io import read_image
from torchvision import transforms
path = "/root/DeepAccident/data/DeepAccident_data/type1_subtype1_accident/ego_vehicle/Camera_Front/Town01_type001_subtype0001_scenario00008"
img_name = "{}_{:03d}.jpg".format("Town01_type001_subtype0001_scenario00008",1)
img_path = os.path.join(path,img_name)
img = read_image(img_path)/255
transform = transforms.Compose([
            transforms.Resize((224, 224)),     
        ])
sample = transform(img).unsqueeze(0)

In [5]:
latent = vae.encode(sample)['latent_dist'].mode()
rec = vae.decoder(latent)

In [6]:
latent.shape

torch.Size([1, 4, 28, 28])

In [37]:
from torchvision.utils import save_image
save_image(rec.data[0],"./test.png")